<a href="https://colab.research.google.com/github/vedanthshenoy/Sales-Analysis-Prediction-and-Recommendation/blob/main/shop_data_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data=pd.read_csv('/content/drive/MyDrive/shopdata.csv',parse_dates=['InvoiceDate'],encoding= 'unicode_escape',index_col='InvoiceDate')
data.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
InvoiceDate,,,,,,,
2010-12-01 08:26:00,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom
2010-12-01 08:26:00,536365,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom
2010-12-01 08:26:00,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom
2010-12-01 08:26:00,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom
2010-12-01 08:26:00,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom


In [3]:
data['Day']=data.index.day
data['Month']=data.index.month
data['DayOfWeek']=data.index.dayofweek
data['Hour']=data.index.hour

data.drop(columns=['InvoiceNo', 'StockCode', 'UnitPrice'],inplace=True)
data=data.dropna(how='any')

data.head()

,Description,Quantity,CustomerID,Country,Day,Month,DayOfWeek,Hour
InvoiceDate,,,,,,,,
2010-12-01 08:26:00,WHITE HANGING HEART T-LIGHT HOLDER,6,17850.0,United Kingdom,1,12,2,8
2010-12-01 08:26:00,WHITE METAL LANTERN,6,17850.0,United Kingdom,1,12,2,8
2010-12-01 08:26:00,CREAM CUPID HEARTS COAT HANGER,8,17850.0,United Kingdom,1,12,2,8
2010-12-01 08:26:00,KNITTED UNION FLAG HOT WATER BOTTLE,6,17850.0,United Kingdom,1,12,2,8
2010-12-01 08:26:00,RED WOOLLY HOTTIE WHITE HEART.,6,17850.0,United Kingdom,1,12,2,8


Considering Spain ,because it is having a fairly good number of sales.

In [4]:
data=data.loc[data['Country'] == 'Spain']

In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['Description'] = le.fit_transform(data['Description'])

data.head()

,Description,Quantity,CustomerID,Country,Day,Month,DayOfWeek,Hour
InvoiceDate,,,,,,,,
2010-12-03 12:20:00,531,70,12557.0,Spain,3,12,4,12
2010-12-03 12:20:00,528,100,12557.0,Spain,3,12,4,12
2010-12-03 12:20:00,521,60,12557.0,Spain,3,12,4,12
2010-12-03 12:20:00,529,70,12557.0,Spain,3,12,4,12
2010-12-03 12:20:00,524,100,12557.0,Spain,3,12,4,12


In [6]:
data.drop(columns=['Country'],inplace=True)

In [7]:
target=data[['Description','Quantity']]
data.drop(columns=['Description','Quantity'],inplace=True)

In [8]:
train_size=int(len(data)*0.9)
x_train,x_test=data[:train_size],data[train_size:len(data)]
y_train,y_test=target[:train_size],target[train_size:len(data)]
x_train.shape,y_train.shape

((2279, 5), (2279, 2))

In [9]:
type(x_train)

pandas.core.frame.DataFrame

In [10]:
import numpy as np
def create_dataset(X,y, time_steps=1):
    dataX, dataY = [], []
    for i in range(len(X) - time_steps - 1):
        a = X.iloc[i:(i + time_steps)].values
        dataX.append(a)
    for i in range(len(y) - time_steps-1):
        dataY.append(y.iloc[i:(i + time_steps)].values)
    return np.array(dataX), np.array(dataY)

In [11]:
time_steps=50

x_train, y_train = create_dataset(x_train, y_train, time_steps)
x_test, y_test = create_dataset(x_test, y_test, time_steps)
x_train.shape,x_test.shape

((2228, 50, 5), (203, 50, 5))

In [12]:
y_train.shape

(2228, 50, 2)

In [13]:
from sklearn.preprocessing import MinMaxScaler

rs_data = MinMaxScaler(feature_range=(0,3))
rs_target = MinMaxScaler(feature_range=(0,2))

# target=data[['Description','Quantity']]
# data.drop(columns=['Description','Quantity'],inplace=True)

nsamples, nx, ny = x_train.shape
x_train = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
x_test = x_test.reshape((nsamples,nx*ny))

nsamples, nx, ny = y_train.shape
y_train = y_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = y_test.shape
y_test = y_test.reshape((nsamples,nx*ny))

x_train=rs_data.fit_transform(x_train)
x_test=rs_data.fit_transform(x_test)
y_train=rs_target.fit_transform(y_train)
y_test=rs_target.fit_transform(y_test)

nsamples, nx = x_train.shape
x_train = x_train.reshape((nsamples,50,5))     #reshape accordingly to your project

nsamples, nx = x_test.shape
x_test = x_test.reshape((nsamples,50,5))

nsamples, nx = y_train.shape
y_train = y_train.reshape((nsamples,50,2))

nsamples, nx = y_test.shape
y_test = y_test.reshape((nsamples,50,2))

In [14]:
y_train.shape

(2228, 50, 2)

In [15]:
import keras
model = keras.Sequential()
model.add(keras.layers.Conv1D(filters=32, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(
  keras.layers.Bidirectional(
    keras.layers.LSTM(
      units=128,return_sequences=True
    )
  ))
model.add(
  keras.layers.Bidirectional(
  keras.layers.LSTM(
  units=500,return_sequences=True
    )
  ))
model.add(keras.layers.Bidirectional(
    keras.layers.LSTM(
  units=500,return_sequences=True)
))
model.add(keras.layers.Dropout(rate=0.3))
model.add(keras.layers.Dense(units=100))
model.add(keras.layers.Dense(units=10))
model.add(keras.layers.Dense(units=2))
model.compile(loss='mean_squared_error', optimizer='adam')

In [16]:
import tensorflow
config = tensorflow.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
tensorflow.compat.v1.keras.backend.set_session(tensorflow.compat.v1.Session(config=config))

In [17]:
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=800,
    validation_split=0.35,
    shuffle=False,
    verbose=1
)

Epoch 1/10
2/2 [==============================] - 4s 2s/step - loss: 9.5539 - val_loss: 1.1761
Epoch 2/10
2/2 [==============================] - 3s 1s/step - loss: 1.3264 - val_loss: 0.9490
Epoch 3/10
2/2 [==============================] - 3s 1s/step - loss: 0.6666 - val_loss: 0.4935
Epoch 4/10
2/2 [==============================] - 3s 1s/step - loss: 0.4404 - val_loss: 0.5916
Epoch 5/10
2/2 [==============================] - 3s 1s/step - loss: 0.3112 - val_loss: 0.2584
Epoch 6/10
2/2 [==============================] - 3s 1s/step - loss: 0.3717 - val_loss: 0.3472
Epoch 7/10
2/2 [==============================] - 3s 1s/step - loss: 0.3669 - val_loss: 0.2161
Epoch 8/10
2/2 [==============================] - 3s 1s/step - loss: 0.2479 - val_loss: 0.3498
Epoch 9/10
2/2 [==============================] - 3s 1s/step - loss: 0.2822 - val_loss: 0.3956
Epoch 10/10
2/2 [==============================] - 3s 1s/step - loss: 0.2626 - val_loss: 0.2277


In [18]:
y_train.shape

(2228, 50, 2)

In [58]:
pred=model.predict(x_test)


nsamples1, nx1, ny1 = pred.shape
pred = pred.reshape((nsamples1,nx1*ny1))

pred=rs_target.inverse_transform(pred)

nsamples1, nx1 = pred.shape
pred = pred.reshape((nsamples1,50,2))

In [59]:
for i in pred:
  for j in i:
    j[0]=round(j[0])
    j[1]=round(j[1])

Get predictionsfor which item would be sold and how much will be sold in the next 50 days

In [60]:
pred.astype(np.int32)

array([[[195,  60],
        [239,  73],
        [278,  85],
        ...,
        [326, 114],
        [277,  98],
        [216,  79]],

       [[197,  61],
        [242,  74],
        [282,  87],
        ...,
        [325, 114],
        [276,  98],
        [215,  78]],

       [[200,  62],
        [245,  76],
        [287,  89],
        ...,
        [324, 114],
        [275,  98],
        [214,  78]],

       ...,

       [[344, 108],
        [445, 132],
        [531, 152],
        ...,
        [524, 146],
        [454, 124],
        [362,  98]],

       [[350, 110],
        [452, 134],
        [537, 154],
        ...,
        [524, 146],
        [454, 124],
        [362,  98]],

       [[350, 110],
        [452, 134],
        [537, 154],
        ...,
        [524, 146],
        [454, 124],
        [362,  98]]], dtype=int32)

In [69]:
integer_mapping = {i: l for i, l in enumerate(le.classes_)}
print(integer_mapping.get(195))
print(integer_mapping.get(239))
print(integer_mapping.get(278))

CHILLI LIGHTS
DAIRY MAID TOASTRACK
DOORMAT WELCOME PUPPIES


EXERCISE : Train the model for 1 day and get the next day's single prediction by providing a single row values instead of x_test in model.predict().